In [22]:
import utils
import loss
import models
import numpy as np
from tensorflow import keras
from matplotlib import pyplot as plt
from sklearn.model_selection import train_test_split

In [2]:
X, Y, index = utils.load_data("data/labels/lcwaikiki_labels.csv", resize = (300, 400))

100%|██████████| 9995/9995 [00:26<00:00, 372.26it/s]


In [3]:
X.shape

(9987, 400, 300, 3)

In [ ]:
# Generate triplets
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size = 0.2, random_state = 42)
X_train = generate_triplet(X_train, Y_train, ap_pairs=NUM_PAIRS, an_pairs=NUM_PAIRS)

In [19]:
INPUT_SHAPE = (400, 300, 3)
FEATURE_SIZE = 64

In [20]:
# Prepare network
anchor_input = keras.layers.Input(INPUT_SHAPE, name='anchor_input')
positive_input = keras.layers.Input(INPUT_SHAPE, name='positive_input')
negative_input = keras.layers.Input(INPUT_SHAPE, name='negative_input')

# Shared embedding layer for positive and negative items
Shared_DNN = models.minixception(INPUT_SHAPE, feature_size = FEATURE_SIZE)

# Individual outputs
encoded_anchor = Shared_DNN(anchor_input)
encoded_positive = Shared_DNN(positive_input)
encoded_negative = Shared_DNN(negative_input)

# Merged output layer
merged_vector = keras.layers.concatenate([encoded_anchor, encoded_positive, encoded_negative], axis=-1, name='merged_layer')

# Define optimizer
adam_optim = keras.optimizers.Adam(lr=0.0001, beta_1=0.9, beta_2=0.999)

# Setup and compile model
model = keras.models.Model(inputs=[anchor_input,positive_input, negative_input], outputs=merged_vector)
model.compile(loss=loss.triplet_loss, optimizer=adam_optim)

In [21]:
model.summary()

Model: "model_9"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
anchor_input (InputLayer)       [(None, 400, 300, 3) 0                                            
__________________________________________________________________________________________________
positive_input (InputLayer)     [(None, 400, 300, 3) 0                                            
__________________________________________________________________________________________________
negative_input (InputLayer)     [(None, 400, 300, 3) 0                                            
__________________________________________________________________________________________________
model_8 (Model)                 (None, 64)           61224       anchor_input[0][0]               
                                                                 positive_input[0][0]       